# Data Generation

This tutorial is the first step in implementing **Constitutional AI** techniques in the context of education. The objective is to demonstrate how to generate a foundational dataset that aligns with a "constitution" of principles aimed at guiding AI behavior. These principles ensure the AI serves as a learning aid, promoting understanding without substituting for students' effort or giving direct answers.

The approach involves:
1. **Generating prompts and initial responses**: Using an AI model to simulate potential student interactions.
2. **Critiquing and revising**: Guiding the AI to critique and improve its responses according to a defined set of principles.
3. **Creating a refined dataset**: Compiling the revised responses into a dataset for fine-tuning an AI model that aligns with educational goals.

To run this procedure on GPU can be done running the [generate_dataset.py](./generate_dataset.py) script.

In [1]:
import json
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

## Load the LLM

This section demonstrates how to load a pre-trained Mistral-7B Instruct model from Hugging Face and configure it for efficient inference. 

We will define a function for generating text based on a user-provided prompt.

In [2]:
if torch.cuda.is_available():
    # Use 8-bit quantization to reduce memory usage while maintaining performance
    quantization_config = BitsAndBytesConfig(load_in_8bit=True)
else:
    # No quantization for systems without CUDA support
    quantization_config = None

# Specify the pre-trained model to load from Hugging Face
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load the tokenizer for the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the pre-trained model with automatic device placement and quantization if applicable
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto",  # Automatically determine device mapping (e.g., CPU or GPU)
    quantization_config=quantization_config, 
)

# Add special tokens to the tokenizer (important for handling special cases during generation)
tokenizer.add_special_tokens({
    "sep_token": "", 
    "cls_token": "", 
    "mask_token": "",  
    "pad_token": "[PAD]"
})

# Define a function for generating text responses based on a user-provided prompt
def generate_text(prompt):
    """
    Generates text using the loaded Mistral model given a prompt.
    
    Args:
        prompt (str): The input text for which the model generates a response.
    
    Returns:
        tuple: Generated response as a string and the number of tokens in the response.
    """
    # Tokenize the input prompt for the model
    inputs = tokenizer(prompt, return_tensors="pt", padding=False)

    # Generate text based on the input prompt
    outputs = model.generate(
        inputs["input_ids"],  # Tokenized input IDs
        attention_mask=inputs["attention_mask"],  # Attention mask for the input
        pad_token_id=tokenizer.eos_token_id,  # Specify the padding token ID
        max_new_tokens=800,  # Maximum number of tokens to generate
        temperature=1.0,  # Sampling temperature (controls randomness)
        do_sample=True  # Enable sampling for more varied responses
    )

    # Extract the response by removing the input tokens from the output
    prompt_length = inputs['input_ids'].shape[1]  # Length of the input prompt
    outputs = outputs[0][prompt_length:]  # Remove input tokens from the output
    num_tokens = len(outputs)  # Count the number of tokens in the response

    # Decode the response into a readable string
    response = tokenizer.decode(outputs, skip_special_tokens=True)

    # Clean up the response by removing unnecessary leading characters
    for s in ['\n', ' ', 'A: ', 'Answer: ']:
        response = response.lstrip(s)

    # Return the cleaned response and the token count
    return response, num_tokens

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Load Constitutional Principles and Examples

In this section, we load a dataset containing **constitutional principles** and examples of **prompt-response-critique-correction** interactions. These elements are critical for aligning the model's outputs with the desired educational principles.

### Contents of the Data File (`constitution_education.json`)

1. **Constitutions**: A list of principles designed to guide AI responses. Each principle includes:
   - A **critic** section that evaluates the quality of a response (e.g., whether it promotes critical thinking or encourages independence).
   - A **revision** section that provides guidance for improving the response to better align with the principles.

2. **System Chat**: Examples of user queries, initial responses from the assistant, critiques, and revised responses. These interactions serve as templates for generating data that is both educationally valuable and aligned with the constitution.

The example below illustrates how to load and preprocess this data:

In [3]:
import json

# Load the constitutional principles and examples of prompt-response-critique-corrections
with open("data/constitution_education.json") as f:
    data = json.load(f)
    # Extract constitutional principles
    constitutions = data["constitutions"]
    # Flatten nested examples of system chat
    system_chat = [item for sublist in data["system_chat"] for item in sublist]

In [4]:
constitutions[:3]

[{'critic': 'Does the response inspire the student to think about how they approached the problem? For example, does it ask them to reflect on their reasoning process or consider alternative approaches?',
  'revision': 'Rewrite the response to include thoughtful questions that encourage the student to reflect on their reasoning and approach.'},
 {'critic': 'Think carefully about whether the response handed the solution to the student too easily. What is the strongest argument you can make that the response discouraged independent thinking?',
  'revision': 'Revise the response to focus on guiding the student toward the solution through prompts or hints, without explicitly solving the problem for them.'},
 {'critic': 'Did the response help the student develop problem-solving skills? If not, how could it have supported them in figuring out the solution themselves?',
  'revision': 'Please rewrite the response to include strategies or steps that guide the student in developing their problem

In [5]:
system_chat[:6]

[{'role': 'user',
  'content': 'Can you explain why multiplying -2 × -3 = 6 without asking me questions?'},
 {'role': 'assistant',
  'content': 'Multiplying two negative numbers gives a positive because it follows the rule that reversing a reversal returns to the original direction, making -2 × -3 = 6.'},
 {'role': 'user',
  'content': 'Does the response inspire the student to think about how they approached the problem? For example, does it ask them to reflect on their reasoning process or consider alternative approaches?'},
 {'role': 'assistant',
  'content': 'The response provides the direct explanation but does not encourage the student to reflect on their reasoning process or consider alternative approaches.'},
 {'role': 'user',
  'content': 'Rewrite the response to include thoughtful questions that encourage the student to reflect on their reasoning and approach.'},
 {'role': 'assistant',
  'content': 'Multiplying two negative numbers results in a positive because it follows the 

In [6]:
# Example of using the LLM to generate text
generate_text(prompt=system_chat[2]['content'])

('', 1)

# Load Training and Test Prompts

Load a dataset of training, validation, and test prompts.

In [7]:
with open("data/student_prompts.json", "r") as f:
    ds = json.load(f)

In [8]:
ds['train'][0]['prompt']

'Provide a comprehensive analysis of the factors leading to the American Civil Rights Movement.'

## Generate Data Samples with User Prompts and Constitutional Principles

This section defines a function to create a single dataset sample based on a student task, a random constitutional principle, and an initial user-assistant interaction. The function incorporates critiques and revisions into the sample, making it ready for fine-tuning the AI model.

In [9]:
def create_sample(split, i, task):
    """
    Generate a data sample including the initial response, critique, and revision.
    
    Args:
        split (str): The dataset split (e.g., 'train', 'validation', 'test').
        i (int): Index of the task in the dataset.
        task (str): The initial prompt/task provided by the student.
    
    Returns:
        tuple: The split, index, token length, and a dictionary containing the
               initial prompt, response, critique, and revised response.
    """
    # Copy the user-assistant interactions template (system_chat)
    chat = system_chat.copy()
    
    # Select a random constitutional principle for critique and revision
    constitution = random.choice(constitutions)
    
    # Initialize an empty dictionary to store the sample's components
    row = {}

    # Loop through the student prompt, critique, and revision phases
    for prompt, prompt_key, response_key in [
        (task, "init_prompt", "init_response"),  # Initial task-response pair
        (constitution["critic"], "critic_prompt", "critic_response"),  # Critique phase
        (constitution["revision"], "revision_prompt", "revision_response")]:  # Revision phase

        # Add the current prompt (user input) to the conversation
        prompt_dict = {"role": "user", "content": prompt}
        chat.append(prompt_dict)

        # Generate a response from the LLM for the current prompt
        completion, token_length = generate_text(
            prompt=tokenizer.apply_chat_template(chat, tokenize=False)
        )

        # Add the response (assistant output) to the conversation
        response_dict = {"role": "assistant", "content": completion}
        chat.append(response_dict)

        # Save the prompt and response in the sample dictionary
        row[prompt_key] = prompt
        row[response_key] = completion

    # Return the dataset split, task index, token length, and the sample dictionary
    return split, i, token_length, row

## Test Data Generation

Testing with a single example

In [13]:
split='train'
i = 1
task = ds['train'][i]['prompt']
_,_,_,row = create_sample(split, i, task)
row

{'init_prompt': 'Provide a detailed explanation of the process of cellular respiration and its importance to living organisms.',
 'init_response': 'Cellular respiration is the process by which cells convert glucose into energy, a process vital for the survival and functioning of living organisms. There are two main stages of cellular respiration: glycolysis, which takes place in the cytoplasm of the cell, and the citric acid cycle, which occurs in the mitochondria. During glycolysis, which is anaerobic, glucose is broken down into pyruvate, generating two molecules of ATP and two of NADH, and releasing two molecules of CO2. This process can be completed without oxygen. In the citric acid cycle, which is aerobic, the two pyruvate molecules enter the Krebs cycle and undergo further breakdown, producing ATP, NADH, and FADH2. This process occurs in the presence of oxygen, which is essential for producing ATP through oxidative phosphorylation.\n\nThe process of cellular respiration is cruci

## Create dataset

Use the function to process the entire dataset.

In [ ]:
#tasks = [process_text(split, idx, row["prompt"]) for split in ds for idx, row in enumerate(ds[split])]